In [70]:
from qoop.core import state, ansatz, metric
import qiskit
from qiskit import transpile
from qiskit.quantum_info import Operator, DensityMatrix, Kraus
from scipy.linalg import qr
import numpy as np
import tensorflow as tf

In [71]:
# Step 1: Read quantum compilation and learn how to use qoop
# https://github.com/vutuanhai237/qoop/wiki/Advance:-Custom-state-preparation

# Step 2: Implement the following function

def V(num_qubits: int):
    '''
        Create a ansatz V with n qubits and assign its parameters
    '''
    #Assign random parameter
    circuit = ansatz.stargraph(num_qubits=num_qubits)
    num_params = circuit.num_parameters
    x0 = 2 * np.pi * np.random.random(num_params)
    circuit = circuit.assign_parameters(dict(zip(circuit.parameters, x0)))
    return circuit

# def Epsilon(rho, kraus_operators):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return sum(K @ rho.data @ np.transpose(np.conjugate(K)) for K in kraus_operators)

# def Epsilon2(rho, unitary_matrix):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return (np.transpose(np.conjugate(unitary_matrix)) @ rho.data @ unitary_matrix)

def calculate_rho_3 (rho, unitary_matrix, kraus_operators):
    '''
        sum(K @ U @ rho @ U(dagger) @ K(dagger))
    '''
    rho_2 = unitary_matrix  @ rho @ np.transpose(np.conjugate(unitary_matrix))
    rho_3 = sum(K @ rho_2 @ np.transpose(np.conjugate(K)) for K in kraus_operators)
    return rho_3

def create_kraus_operators(unitary_matrix):
    '''
        Create a set of Kraus Operators from the input unitary matrix, using QR decomposition
    '''
    kraus_ops = []
    Q, R = qr(unitary_matrix)

    #Q: a 2^N x 2^N matrix, N is the number of qubits
    for q in Q:
        q = np.expand_dims(q, 1)
        kraus_ops.append(q @ np.transpose(np.conjugate(q)))
    return tf.convert_to_tensor(kraus_ops)

def frobenius_norm(rho, sigma):
    """
    Compute the Frobenius norm between two matrices.

    Parameters:
    rho (numpy.ndarray): The first matrix.
    sigma (numpy.ndarray): The second matrix.

    Returns:
    float: The Frobenius norm between rho and rho_prime.
    """
    # Compute the difference between the matrices
    diff = tf.subtract(rho, sigma)
    
    # Compute the Frobenius norm
    norm = tf.linalg.normalize(diff, ord='fro', axis=(0,1))
    
    return norm

#cost func to compare 2 given rhos
def cost(rho, rho_3):
    return frobenius_norm(rho, rho_3) 


In [72]:
n = 3

#Create a ansatz V with N qubits
circuit = V(n)

print(circuit)

     ┌────────────┐   ┌─────────────┐   
q_0: ┤ Ry(5.3425) ├─■─┤ Ry(0.48333) ├─■─
     ├────────────┤ │ └─────────────┘ │ 
q_1: ┤ Ry(4.4346) ├─■─────────────────┼─
     ├────────────┤                   │ 
q_2: ┤ Ry(2.4779) ├───────────────────■─
     └────────────┘                     


In [73]:
# Get the unitary operator corresponding to the circuit
unitary_op = Operator(circuit)

# Get the unitary matrix
unitary_matrix = unitary_op.data

print (unitary_matrix) #2^N x 2^N

[[ 0.1911474 +0.j  0.04449236+0.j  0.25327516+0.j  0.05895351+0.j
  -0.55470483+0.j -0.12911569+0.j -0.73499801+0.j -0.17108157+0.j]
 [-0.04449236+0.j  0.1911474 +0.j -0.05895351+0.j  0.25327516+0.j
   0.12911569+0.j -0.55470483+0.j  0.17108157+0.j -0.73499801+0.j]
 [-0.19686578+0.j -0.16990494+0.j  0.1485751 +0.j  0.12822768+0.j
   0.57129944+0.j  0.49305975+0.j -0.43116111+0.j -0.37211342+0.j]
 [-0.16990494+0.j  0.19686578+0.j  0.12822768+0.j -0.1485751 +0.j
   0.49305975+0.j -0.57129944+0.j -0.37211342+0.j  0.43116111+0.j]
 [ 0.55470483+0.j  0.12911569+0.j  0.73499801+0.j  0.17108157+0.j
   0.1911474 +0.j  0.04449236+0.j  0.25327516+0.j  0.05895351+0.j]
 [ 0.12911569+0.j -0.55470483+0.j  0.17108157+0.j -0.73499801+0.j
   0.04449236+0.j -0.1911474 +0.j  0.05895351+0.j -0.25327516+0.j]
 [-0.57129944+0.j -0.49305975+0.j  0.43116111+0.j  0.37211342+0.j
  -0.19686578+0.j -0.16990494+0.j  0.1485751 +0.j  0.12822768+0.j]
 [ 0.49305975+0.j -0.57129944+0.j -0.37211342+0.j  0.43116111+0.j
   

In [74]:
# Initialize a set of Kraus Operators from the given circuit
kraus_operators = create_kraus_operators(unitary_matrix=unitary_matrix)

#print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperators))
print(unitary_matrix @ np.transpose(np.conjugate(unitary_matrix)))


[[ 1.00000000e+00+0.j -1.03521999e-17+0.j  9.69475925e-18+0.j
  -1.33259756e-17+0.j -1.70363634e-17+0.j  3.97561495e-17+0.j
   8.83563232e-18+0.j  2.76481682e-17+0.j]
 [-1.03521999e-17+0.j  1.00000000e+00+0.j -4.89940579e-17+0.j
   6.76030196e-17+0.j  3.24855348e-17+0.j  2.35434328e-17+0.j
  -6.64655894e-18+0.j  9.08907893e-18+0.j]
 [ 9.69475925e-18+0.j -4.89940579e-17+0.j  1.00000000e+00+0.j
  -2.82492892e-18+0.j  7.73513295e-18+0.j -1.08294481e-17+0.j
  -1.92178881e-17+0.j -2.80311529e-17+0.j]
 [-1.33259756e-17+0.j  6.76030196e-17+0.j -2.82492892e-18+0.j
   1.00000000e+00+0.j  2.38723360e-17+0.j  1.28523874e-17+0.j
  -3.41262757e-17+0.j -4.75947261e-17+0.j]
 [-1.70363634e-17+0.j  3.24855348e-17+0.j  7.73513295e-18+0.j
   2.38723360e-17+0.j  1.00000000e+00+0.j  1.07366633e-17+0.j
  -1.51500818e-18+0.j  6.17814688e-17+0.j]
 [ 3.97561495e-17+0.j  2.35434328e-17+0.j -1.08294481e-17+0.j
   1.28523874e-17+0.j  1.07366633e-17+0.j  1.00000000e+00+0.j
   3.75826938e-17+0.j  2.26178793e-17+0.j

In [75]:
#Initialize rho
rho = DensityMatrix.from_label('0' * n)
rho = tf.convert_to_tensor(rho)

#calculate rho_3
rho_3 = calculate_rho_3(rho=rho, kraus_operators=kraus_operators, unitary_matrix=unitary_matrix) 

#print(rho)
#print(rho3)


In [76]:
print(cost(rho=rho, rho_3=rho_3))

(<tf.Tensor: shape=(8, 8), dtype=complex128, numpy=
array([[ 8.10539363e-01+0.j,  2.95742934e-02+0.j, -2.49726932e-02+0.j,
         3.80729081e-02+0.j,  5.19664056e-02+0.j, -2.38227014e-02+0.j,
        -7.27372641e-02+0.j,  8.83742087e-03+0.j],
       [ 2.95742934e-02+0.j, -5.55134423e-02+0.j, -3.80729081e-02+0.j,
        -2.40984511e-02+0.j,  2.38227014e-02+0.j,  3.65904319e-03+0.j,
        -8.83742087e-03+0.j,  5.75134000e-03+0.j],
       [-2.49726932e-02+0.j, -3.80729081e-02+0.j, -8.95523216e-02+0.j,
        -7.86044027e-03+0.j,  7.27372641e-02+0.j, -8.83742087e-03+0.j,
        -9.34501364e-02+0.j,  2.88628853e-02+0.j],
       [ 3.80729081e-02+0.j, -2.40984511e-02+0.j, -7.86044027e-03+0.j,
        -4.17695426e-02+0.j, -8.83742087e-03+0.j,  5.75134000e-03+0.j,
         2.88628853e-02+0.j,  3.78922008e-04+0.j],
       [ 5.19664056e-02+0.j,  2.38227014e-02+0.j,  7.27372641e-02+0.j,
        -8.83742087e-03+0.j, -2.08207736e-01+0.j,  3.13494376e-02+0.j,
         2.10989612e-01+0.j, -6.06

In [77]:
# Step 3: Implement the following function
# state_need_tomogaphy = ...
# rho = np.conjugate(np.transpose(state_need_tomogaphy)) @ state_need_tomogaphy   # density matrix
# rho' = Delta(rho)
# compiler = qoop.qcompilation.QuantumCompilation(U = rho', V = V())
# compiler.fit()
# compiler.plot()
# see fidelities versus iteration

In [78]:
#Auto Diff
'''
    rho: The initial rho
    unitary_matrix: randomly generated circle in initialization step
    kraus_operators: set of kraus operators
    n: number of qubits
    alpha: learning rate (?)'''
def calculate_derivative(rho, unitary_matrix, kraus_operators, n, alpha=0.1):
    tensorKraus = tf.Variable(kraus_operators)
    with tf.GradientTape() as tape:
        y = calculate_rho_3(rho, unitary_matrix, tensorKraus)
        f = 1 - compilation_trace_fidelity(rho, y) 
    
    # Get the gradient of y with respect to x
    c = tape.gradient(f, tensorKraus)

    # Reshape c to kN * N matrix
    c = tf.reshape(c, (2**n * 2**n,2**n))

    # Reshape kraus_operators to kN * N matrix
    kraus_operators = tf.reshape(kraus_operators, (2**n * 2**n,2**n))
    
    # Compute the projection term
    proj = c - kraus_operators @ (np.transpose(np.conjugate(c)) @ kraus_operators  + np.transpose(np.conjugate(kraus_operators)) @ c) / 2

    # Update the Kraus operators
    updated_kraus_operators = kraus_operators - alpha * proj
    
    # Return the updated Kraus operators
    return updated_kraus_operators



In [79]:
kraus_operators_copy = tf.identity(kraus_operators)

In [80]:
# try looping manually
for i in range (0, 1000):
    _cost = cost(rho, calculate_rho_3(rho, unitary_matrix, kraus_operators_copy))

    #Update Kraus Operators
    kraus_operators_copy=calculate_derivative(rho, unitary_matrix, kraus_operators_copy, n)

    #Reshape
    kraus_operators_copy = tf.reshape(kraus_operators_copy, (2**n,2**n,2**n))
    
    #print('K(t)K\n',sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry)) # I

    print (_cost)

(<tf.Tensor: shape=(8, 8), dtype=complex128, numpy=
array([[ 8.10539363e-01+0.j,  2.95742934e-02+0.j, -2.49726932e-02+0.j,
         3.80729081e-02+0.j,  5.19664056e-02+0.j, -2.38227014e-02+0.j,
        -7.27372641e-02+0.j,  8.83742087e-03+0.j],
       [ 2.95742934e-02+0.j, -5.55134423e-02+0.j, -3.80729081e-02+0.j,
        -2.40984511e-02+0.j,  2.38227014e-02+0.j,  3.65904319e-03+0.j,
        -8.83742087e-03+0.j,  5.75134000e-03+0.j],
       [-2.49726932e-02+0.j, -3.80729081e-02+0.j, -8.95523216e-02+0.j,
        -7.86044027e-03+0.j,  7.27372641e-02+0.j, -8.83742087e-03+0.j,
        -9.34501364e-02+0.j,  2.88628853e-02+0.j],
       [ 3.80729081e-02+0.j, -2.40984511e-02+0.j, -7.86044027e-03+0.j,
        -4.17695426e-02+0.j, -8.83742087e-03+0.j,  5.75134000e-03+0.j,
         2.88628853e-02+0.j,  3.78922008e-04+0.j],
       [ 5.19664056e-02+0.j,  2.38227014e-02+0.j,  7.27372641e-02+0.j,
        -8.83742087e-03+0.j, -2.08207736e-01+0.j,  3.13494376e-02+0.j,
         2.10989612e-01+0.j, -6.06

In [81]:
print(sum(K @ np.transpose(np.conjugate(K)) for K in kraus_operators_copy))
print(kraus_operators_copy)

tf.Tensor(
[[ 1.79386730e+00+0.00000000e+00j  9.15486683e-02-4.51848830e-20j
  -3.52928626e-02+1.63928487e-19j  1.14058586e-01-7.00441610e-20j
  -5.37460623e-02+1.03824554e-19j -3.66101807e-02+1.65357720e-19j
  -5.07601164e-03+1.92395949e-19j -3.60469262e-02-2.84387053e-19j]
 [ 9.15486683e-02+4.51848830e-20j  9.23038725e-01+0.00000000e+00j
  -5.78499255e-02-7.23130504e-28j -3.94714366e-02+1.38807673e-28j
   2.65201628e-02-1.26423828e-27j  1.23612511e-02+1.76717786e-27j
   8.49498986e-04-5.05178708e-28j  1.08122017e-02+1.37198114e-27j]
 [-3.52928626e-02-1.63928487e-19j -5.78499255e-02+7.23130504e-28j
   8.84590806e-01+0.00000000e+00j -1.68796179e-02+8.86560595e-28j
   5.74441298e-02-2.34202516e-27j  2.19068858e-03+1.55918214e-27j
  -6.24009747e-02-4.89841705e-27j  2.71176883e-02+2.11695714e-27j]
 [ 1.14058586e-01+7.00441610e-20j -3.94714366e-02-1.38807673e-28j
  -1.68796179e-02-8.86560595e-28j  9.49322571e-01+0.00000000e+00j
   1.41147823e-03+2.09245662e-29j  1.01057258e-02+8.46031576e-

In [82]:
#Check the output of sum(K @ U @ rho @ U(dagger) @ K(dagger))  - suppose to be rho
rho_out_1 = calculate_rho_3(rho=rho, unitary_matrix=unitary_matrix, kraus_operators=kraus_operators_copy)

In [85]:
#Compare with rho
def compilation_trace_fidelity(rho, sigma):
    """Calculating the fidelity metric

    Args:
        - rho (DensityMatrix): first density matrix
        - sigma (DensityMatrix): second density matrix

    Returns:
        - float: trace metric has value from 0 to 1
    """
    rho_2 = tf.linalg.sqrtm((tf.linalg.sqrtm(rho)) @ (rho))

    # Cast to a supported type
    real_part = tf.math.real(rho_2)
    imaginary_part = tf.math.imag(rho_2)

    # Check for NaNs in both real and imaginary parts
    contains_nan_real = tf.reduce_any(tf.math.is_nan(real_part))
    contains_nan_imag = tf.reduce_any(tf.math.is_nan(imaginary_part))

    contains_nan = contains_nan_real or contains_nan_imag

    if contains_nan == True:
        rho_2 = rho

    return tf.linalg.trace(
            rho_2
            @ (tf.linalg.sqrtm(sigma))
        )

print(compilation_trace_fidelity(rho,rho_out_1))

tf.Tensor((0.9999999999999999+2.346167390893111e-36j), shape=(), dtype=complex128)


In [84]:
print (rho_out_1)

tf.Tensor(
[[ 1.00000000e+00+1.00063957e-36j -1.83533744e-15-3.77016029e-20j
   1.21972744e-16+1.80977940e-19j  9.65156774e-15-6.73794882e-20j
  -2.60316942e-16+1.04936910e-19j -6.27937372e-15+1.63402769e-19j
   1.89084859e-16+2.12096769e-19j -4.06684235e-15-2.90796567e-19j]
 [-1.83184398e-15+3.77016029e-20j  2.63549248e-18+1.65904431e-37j
   3.54312612e-19-3.26598980e-34j  2.58745000e-19+4.86873359e-34j
   1.15235567e-18-2.01608890e-34j -5.46170812e-18-5.35526892e-34j
   2.58091588e-18-3.81687959e-34j  2.70516494e-18+3.79716483e-34j]
 [ 1.20637073e-16-1.80977940e-19j  2.13227831e-18+3.26430750e-34j
  -3.75819305e-18+2.18627606e-36j  7.81092291e-19-1.75498254e-33j
   3.09764519e-18+5.72841213e-35j  7.05812424e-19+1.15413835e-33j
   4.13636348e-18-3.99750658e-36j  1.32774988e-20+6.99799164e-34j]
 [ 9.64851432e-15+6.73794882e-20j -1.32270962e-18-4.87362784e-34j
   2.53039462e-18+1.75396161e-33j  4.70984693e-18-9.54733889e-37j
  -8.09441787e-19+9.94017004e-34j  2.16066867e-18+1.15429761e-